In [108]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
import requests
import json
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
%matplotlib inline

In [2]:
df = pd.read_csv("policing/Racial_Profiling_Dataset_2015_Citations.csv")

In [3]:
df.head()

,CITATION NUMBER,OFF FROM DATE,OFF TIME,RACE-ORIGIN CODE,CASE PARTY SEX,REASON FOR STOP,RACE KNOWN,VL STREET NAME,MSEARCH Y/N,MSEARCH TYPE,MSEARCH FOUND
0,1111.0,05/25/2015 12:00:00 AM,8:00:00,W,M,1.0,N,10000 BLOCK WEST BEN WHITE BOULEVARD,N,NaN,NaN
1,320540.0,12/07/2015 12:00:00 AM,23:46:00,H,F,1.0,N,SOUTH PLEASANT VALLEY ROAD,N,NaN,NaN
2,1394195.0,09/01/2015 12:00:00 AM,21:05:00,W,M,1.0,N,1300 BLOCK SOUTH 1ST STREET,N,NaN,NaN
3,1395858.0,02/04/2015 12:00:00 AM,20:51:00,W,F,1.0,N,EAST 7TH STREET,N,NaN,NaN
4,1409057.0,12/14/2015 12:00:00 AM,15:57:00,W,M,1.0,N,7200 BLOCK FM 2222 ROAD,N,NaN,NaN


In [122]:
def time_of_day(x):
    hour = int(x["OFF TIME"].split(":")[0])
    if hour > 5 and hour <= 9:
        return "morning"
    if hour > 9 and hour <= 17:
        return "day"
    if hour > 17 and hour < 24:
        return "night"
    if hour <= 5:
        return "night"

def to_hour(x):
    return int(x.split(":")[0])

map_race = {
    "B": 1,
    "W": 2,
    "H": 3,
    "O": 4,
    "ME": 5,
    "A": 6,
    "N": 7
}

is_minority_map = {
    "W": 0,
    "B": 1,
    "H": 1,
    "O": 1,
    "ME": 1,
    "A": 1,
    "N": 1
}

map_gender = {
    "M": 0,
    "F": 1
}

    
#df["gender"] = df["CASE PARTY SEX"].map(map_gender)
#df["is_minority"] = df["RACE-ORIGIN CODE"].map(is_minority_map)
#df["to_hour"] = df["OFF TIME"].apply(to_hour)

{'results': [{'address_components': [{'long_name': 'West Ben White Boulevard', 'short_name': 'W Ben White Blvd', 'types': ['route']}, {'long_name': 'Austin', 'short_name': 'Austin', 'types': ['locality', 'political']}, {'long_name': 'Travis County', 'short_name': 'Travis County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Texas', 'short_name': 'TX', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}], 'formatted_address': 'W Ben White Blvd, Austin, TX, USA', 'geometry': {'bounds': {'northeast': {'lat': 30.2356182, 'lng': -97.7525352}, 'southwest': {'lat': 30.21494, 'lng': -97.79737089999999}}, 'location': {'lat': 30.2272657, 'lng': -97.7753755}, 'location_type': 'GEOMETRIC_CENTER', 'viewport': {'northeast': {'lat': 30.2356182, 'lng': -97.7525352}, 'southwest': {'lat': 30.21494, 'lng': -97.79737089999999}}}, 'place_id': 'ChIJZ5ADDsq0RIYRNHjidUg4WGc', 'types': ['route']}

KeyboardInterrupt: 

In [124]:
lats, longs

([30.2272657, 30.2067971], [-97.7753755, -97.74041079999999])

In [36]:
df.head()

,CITATION NUMBER,OFF FROM DATE,OFF TIME,RACE-ORIGIN CODE,CASE PARTY SEX,REASON FOR STOP,RACE KNOWN,VL STREET NAME,MSEARCH Y/N,MSEARCH TYPE,MSEARCH FOUND,gender,is_minority,to_hour
0,1111.0,05/25/2015 12:00:00 AM,8:00:00,W,M,1.0,N,10000 BLOCK WEST BEN WHITE BOULEVARD,N,NaN,NaN,0.0,1.0,8
1,320540.0,12/07/2015 12:00:00 AM,23:46:00,H,F,1.0,N,SOUTH PLEASANT VALLEY ROAD,N,NaN,NaN,1.0,1.0,23
2,1394195.0,09/01/2015 12:00:00 AM,21:05:00,W,M,1.0,N,1300 BLOCK SOUTH 1ST STREET,N,NaN,NaN,0.0,1.0,21
3,1395858.0,02/04/2015 12:00:00 AM,20:51:00,W,F,1.0,N,EAST 7TH STREET,N,NaN,NaN,1.0,1.0,20
4,1409057.0,12/14/2015 12:00:00 AM,15:57:00,W,M,1.0,N,7200 BLOCK FM 2222 ROAD,N,NaN,NaN,0.0,1.0,15


In [31]:
df["RACE-ORIGIN CODE"].unique()

array(['W', 'H', 'B', nan, 'O', 'ME', 'A', 'N'], dtype=object)

In [32]:
# Features:
# latitude, longitude
# wealth of area
# reason for stop
# proximity to school (K-12)
# median income
# proximity to hospital
# proximity to college

110847

In [94]:
subsection = df[["REASON FOR STOP", "to_hour", "is_minority"]]
subsection = subsection.dropna()
X = subsection[["REASON FOR STOP", "to_hour"]]
y = subsection["is_minority"]

decision_tree = tree.DecisionTreeClassifier()
prediction = cross_val_predict(decision_tree, X, y, cv=20)
print(metrics.confusion_matrix(prediction, y))
print(metrics.f1_score(prediction, y))
decision_tree.fit(X, y)
print(decision_tree.classes_)
decision_tree.feature_importances_

[[36740 29799]
 [19043 22983]]
0.4848325035861953
[0. 1.]


array([0.33501433, 0.66498567])

In [8]:
dir(metrics)

['SCORERS',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'accuracy_score',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'auc',
 'average_precision_score',
 'balanced_accuracy_score',
 'base',
 'brier_score_loss',
 'calinski_harabaz_score',
 'check_scoring',
 'classification',
 'classification_report',
 'cluster',
 'cohen_kappa_score',
 'completeness_score',
 'confusion_matrix',
 'consensus_score',
 'coverage_error',
 'davies_bouldin_score',
 'euclidean_distances',
 'explained_variance_score',
 'f1_score',
 'fbeta_score',
 'fowlkes_mallows_score',
 'get_scorer',
 'hamming_loss',
 'hinge_loss',
 'homogeneity_completeness_v_measure',
 'homogeneity_score',
 'jaccard_similarity_score',
 'label_ranking_average_precision_score',
 'label_ranking_loss',
 'log_loss',
 'make_scorer',
 'matthews_corrcoef',
 'mean_absolute_error',
 'mean_squared_error',
 'mean_squared_log_error',
 'median_absol

street=9+Poplar+Court&city=Great+Neck&state=NY


{'result': {'addressMatches': [{'matchedAddress': '9 Poplar Ct, GREAT NECK, NY, 11024',
    'coordinates': {'x': -73.736824, 'y': 40.80877},
    'tigerLine': {'side': 'L', 'tigerLineId': '147686111'},
    'addressComponents': {'fromAddress': '1',
     'toAddress': '99',
     'zip': '11024',
     'city': 'GREAT NECK',
     'preQualifier': '',
     'preDirection': '',
     'preType': '',
     'streetName': 'Poplar',
     'suffixType': 'Ct',
     'suffixDirection': '',
     'suffixQualifier': '',
     'state': 'NY'}}],
  'input': {'address': {'street': '9 Poplar Court',
    'city': 'Great Neck',
    'state': 'NY'},
   'benchmark': {'benchmarkDescription': 'Public Address Ranges - Census 2010 Benchmark',
    'benchmarkName': 'Public_AR_Census2010',
    'isDefault': False,
    'id': '9'}}}}